While watching a Matt Parker video on the average maximum value of two dice rolls, he had a python script that ran almost instantly and supposedly chose the max of 2 dice rolls a million times and averaged them. Here I will try to approach that performance without looking at his code. 

https://www.youtube.com/watch?v=X_DdGRjtwAo

In [1]:
from time import time_ns
from random import randint as ri
import numpy as np

def timing(func):
    # This function shows the execution time of 
    # the function object passed
    def wrap_func(*args, **kwargs):
        t1 = time_ns()
        result = func(*args, **kwargs)
        t2 = time_ns()
        td = (t2 - t1) / 1_000_000_000
        print(f'Function {func.__name__!r} executed in {td:.4f}s')
        return result
    return wrap_func

Let's start off with a basic program to do a million dice rolls with a variable number of sides and variable number of dice.

In [2]:
@timing
def basic_roller(sides, simulations=1_000_000, dice_count=2):
    values = []
    for i in range(simulations):
        cvalues = []
        for roll in range(dice_count):
            cvalues.append(ri(1, sides))
        values.append(max(cvalues))
    avg = np.mean(values)
    print(f"A {sides} sided dice has an average of {avg} \nafter taking the maximum value of {dice_count} dice after {simulations} simulations")
    
basic_roller(20)

A 20 sided dice has an average of 13.828616 
after taking the maximum value of 2 dice after 1000000 simulations
Function 'basic_roller' executed in 1.7580s


Single threaded and inefficient at 1.74s, that's a good start. Let's try a list comprehension.

In [3]:
@timing
def list_comprehension_roller(sides, simulations=1_000_000, dice_count=2):
    avg = np.mean([max([ri(1, sides) for y in range(dice_count)]) 
              for x in range(simulations)])
    print(f"A {sides} sided dice has an average of {avg} \nafter taking the maximum value of {dice_count} dice after {simulations} simulations")

list_comprehension_roller(20)

A 20 sided dice has an average of 13.820662 
after taking the maximum value of 2 dice after 1000000 simulations
Function 'list_comprehension_roller' executed in 1.7610s


Well, that's about the same amount of time... let's try it with numpy. 

In [4]:
@timing
def numpy_random_array(sides, simulations=1_000_000, dice_count=2):
    rand_array = np.random.randint(1, sides, size=(simulations, dice_count))
    maxes = rand_array.max(axis=1)
    avg = np.mean(maxes)
    print(f"A {sides} sided dice has an average of {avg} \nafter taking the maximum value of {dice_count} dice after {simulations} simulations")

    
numpy_random_array(20)

A 20 sided dice has an average of 13.160772 
after taking the maximum value of 2 dice after 1000000 simulations
Function 'numpy_random_array' executed in 0.0250s


So that's super fast, let's see if we can do it in a single line.

In [5]:
@timing
def numpy_random_array_sl(sides, simulations=1_000_000, dice_count=2):
    avg = np.mean(np.random.randint(1, sides, size=(simulations, dice_count)).max(axis=1))
    print(f"A {sides} sided dice has an average of {avg} \nafter taking the maximum value of {dice_count} dice after {simulations} simulations")

    
numpy_random_array(20)

A 20 sided dice has an average of 13.165284 
after taking the maximum value of 2 dice after 1000000 simulations
Function 'numpy_random_array' executed in 0.0250s


So, about the same. 
Now let's look at Matt Parker's code from his GitHub linked in the video description. https://github.com/standupmaths/higher_of_two_rolls/blob/main/higher-of-two-rolls.py

In [6]:
import random

number_of_sides  =  int(input("How many sides on your dice? "))
t1 = time_ns()
sum_of_results = 0.0

trials = 0
while trials < 10**6:
    sum_of_results += max([int(random.random()*number_of_sides)+1,int(random.random()*number_of_sides)+1])
    trials += 1

print("Average result of rolling two and taking the highest is about {0}".format(sum_of_results/trials))
t2 = time_ns()
td = (t2 - t1) / 1_000_000_000
print(f'Executed in {td:.4f}s')

How many sides on your dice? 20
Average result of rolling two and taking the highest is about 13.830853
Executed in 0.7080s


I added some timing into the script, and we can see that his is slightly slower. 